In [1]:
import datetime
import json
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit_experiments.framework import ExperimentEncoder
from qiskit_research.mzm_generation import (
    KitaevHamiltonianExperiment,
    KitaevHamiltonianExperimentParameters,
)
from qiskit_research.mzm_generation.utils import transpile_circuit
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations

from qiskit.circuit.library import RZGate
from qiskit_research.utils.dynamical_decoupling import get_instruction_durations

from qiskit_experiments.framework import BatchExperiment

In [2]:
# get backend

# uncomment to use hardware backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend_name = 'ibmq_guadalupe'

# Aer simulator backend
# provider = None
# backend_name = "aer_simulator"

In [3]:
# experiment parameters
n_modes = [9]
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = list(np.linspace(0.0, 3.0, num=5))
# dynamical_decoupling_sequences = None
dynamical_decoupling_sequences=[None, 'X2pm', 'XY4pm', 'XY8pm']
num_dd_passes=1
shots = 10000
readout_calibration_shots = 10000

experiments = []

for n in n_modes:
    # pick qubits
    qubits, backend_name, error_score = pick_qubit_layout(n, backend_name, provider)
    print(f"n = {n} Using qubits {qubits}.")

    occupied_orbitals_list = list(orbital_combinations(n, threshold=2))

    # date
    date = datetime.datetime.now().isoformat()

    # create experiment
    params = KitaevHamiltonianExperimentParameters(
        timestamp=date,
        backend_name=backend_name,
        qubits=qubits,
        n_modes=n,
        tunneling_values=tunneling_values,
        superconducting_values=superconducting_values,
        chemical_potential_values=chemical_potential_values,
        occupied_orbitals_list=occupied_orbitals_list,
        dynamical_decoupling_sequences=dynamical_decoupling_sequences,
        num_dd_passes=num_dd_passes,
        seed=11364,
    )
    experiment = KitaevHamiltonianExperiment(params, provider)
    experiments.append(experiment)

    print(f"Total number of circuits for n = {n}: {len(experiment.circuits())}")

n = 9 Using qubits [2, 3, 5, 8, 11, 14, 13, 12, 15].
Total number of circuits for n = 9: 2520


In [4]:
%%time
# schedule readout calibration
for experiment in experiments:
    n = experiment.params.n_modes
    print("Scheduling readout calibration n = {n}...")
    mit = mthree.M3Mitigation(experiment.backend)
    mit.cals_from_system(experiment.params.qubits, shots=readout_calibration_shots, async_cal=True)

    # schedule experiment jobs
    print(f"Scheduling experiment n = {n}...")
    # limit number of circuits per job to avoid timeouts
    experiment.backend.configuration().max_experiments = 300
    data = experiment.run(shots=shots)
    # if provider:
        # this only works if the Qiskit Experiments database service is available
        # data.save()

    # save readout calibration
    filename = f"data/readout_calibration/{experiment.backend.name()}/{date}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print(f"Waiting for readout calibration data ...")
    mit.cals_to_file(filename)
    print(f"Readout calibration saved.")

    data.block_for_results()

    # save data
    filename = f"data/{params.filename}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        json.dump(data, f, cls=ExperimentEncoder)
    print(params.filename)

Scheduling readout calibration n = {n}...
Scheduling experiment n = 9...
Waiting for readout calibration data ...
Readout calibration saved.


Job was cancelled before completion [Job ID: 629e0faab4d2d3cb5778cb57]
Job was cancelled before completion [Job ID: 629e0fa3a70e338b2e60a76f]
Job was cancelled before completion [Job ID: 629e0fb2c6235224339e3424]
Job was cancelled before completion [Job ID: 629e0f9ca3847125bd452593]


2022-06-06T10:26:51.968588_ibmq_guadalupe_n9
CPU times: user 4min 12s, sys: 6.1 s, total: 4min 18s
Wall time: 4h 9min 28s
